# Color in Digital Imaging
In this notebook, we will look at the basics of color theory, and how color data is stored in digital format.

Since the concepts discussed here are mostly theoretical, this notebook will not contain many practical exercises, and mostly deals with mathematical manipulation of image arrays. If you want to get more familiar with different representations of color images, Chapter 6 of *Digital Image Processing* by *Rafael C. Gonzalez* and *Richard E. Woods* would be a very good source.

## Section 0. Preparing the Notebook
We start by importing the necessary libraries and then loading a sample image to work on.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# importing necessary packages
import numpy as np
from numpy import random
import cv2 as cv
from matplotlib import pyplot as plt
from solutions import *
from utils import *

In [ ]:
# loading the sample image and setting the colormap for pyplot
image = np.float64(cv.imread('data/baboon.bmp', cv.IMREAD_COLOR)) / 255
plt.set_cmap('Greys_r')
_ = plt.imshow(image), plt.axis('off')

Here you might be surprised by seeing a baboon with a bright blue nose. However, this serves as a very good preface to our notebook.

## Section 1. Image Representation

### Section 1.1. Image Representation as the Primary Lights
The most common way of saving color images, is by representing their colors as three channels: Red, green, and blue. The reasoning for this, is that human eyes have three types of color receptors, with each of them sensitive to one of these colors. These are collectively known as the primary lights.

The most common way of ordering these channels is as RGB, i.e., red, green, and blue. However, OpenCV reads color images as BGR, while matplotlib views images in RGB, which is the reason for the Baboon's odd coloring. Try fixing this problem without using OpenCV's ``cvtColor`` function.

In [ ]:
def bgr2rgb(image : np.ndarray) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        A color image in np.ndarray format with dtype=np.float64.
    Returns:
    - output : np.ndarray
        The original image, with channels converted from BGR to RGB.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Converting the image colors
image_RGB = bgr2rgb(image)
image_RGB_ref = bgr2rgbRef(image)

# Showing the result
_ = plt.figure(figsize=(14, 7))
_ = plt.subplot(1, 2, 1), plt.imshow(image_RGB), plt.axis('off'), plt.title('Your RGB Image')
_ = plt.subplot(1, 2, 2), plt.imshow(image_RGB_ref), plt.axis('off'), plt.title('Reference RGB Image')

Sometimes, you might have 4-channel images that are formatted as BGRA or RGBA. There, the A, or alpha channel, represents the *opaqueness* of the image. This is useful for images with transparent parts.

### Section 1.2. Image Representation as the Primary Colors
While the RGB representation is mostly aimed at viewing images in a display, images that are meant to be printed are mostly represented using the CMYK or CMY representation. This representation uses the primary colors, instead of lights. Namely cyan, magenta, and yellow. This is due to the fact that pigments, unlike pixels, create color by absorbing different wavelengths. Therefore, a pigment which absorbs red light will look cyan, one that absorbs green will look magenta, and one that absorbs blue will look yellow.

Knowing this, you can infer that in CMY format, the C value can be interpreted as the negative of the R channel in RGB format, and that the other channels in CMY are also the negatives of their respective channels. Implement a function which converts images from RGB to CMY format.

In [ ]:
def rgb2cmy(image : np.ndarray) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        A color image in np.ndarray format with dtype=np.float64.
    Returns:
    - output : np.ndarray
        The original image, with channels converted from RGB to CMY.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Converting the image colors
image_CMY = rgb2cmy(image_RGB_ref)
image_CMY_ref = rgb2cmyRef(image_RGB_ref)

# Showing the result
_ = plt.figure(figsize=(12, 18))
_ = plt.subplot(1, 2, 1), plt.imshow(np.concatenate([image_CMY[:,:,i] for i in range(3)], axis=0)), plt.axis('off'), plt.title('Your CMY Channels')
_ = plt.subplot(1, 2, 2), plt.imshow(np.concatenate([image_CMY_ref[:,:,i] for i in range(3)], axis=0)), plt.axis('off'), plt.title('Reference CMY Channels')

While this method successfully converts image values to their counterpart color values, CMY is not often used for printing. This is due to the fact that an equal mixture of all CMY pigments, instead of creating a black color, creates a muddy brown, and at the expense of applying three pigments, instead of only using black. Therefore, a more practical approach is using CMYK, with K, or key, representing the value for black. In this approach, the K value represents the share of black color in the image, and the CMY channels only determine the color. The equations for CMY to CMYK conversion are written below. The values for CMYK are marked with an apostrophe to avoid confusion.

$
K' = \min(C,M,Y)
$

$
C' = (C - K) / (1 - K)
$

$
M' = (M - K) / (1 - K)
$

$
Y' = (Y - K) / (1 - K)
$

The exception to this equation is cases where $K=1$, i.e. the color black, in which the other 3 channels will be set to zero.

With this information, implement a conversion function for RGB to CMYK.

In [ ]:
def rgb2cmyk(image : np.ndarray) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        A color image in np.ndarray format with dtype=np.float64.
    Returns:
    - output : np.ndarray
        The original image, with channels converted from RGB to CMYK.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Converting the image colors
image_CMYK = rgb2cmyk(image_RGB_ref)
image_CMYK_ref = rgb2cmykRef(image_RGB_ref)

# Showing the result
_ = plt.figure(figsize=(12, 24))
_ = plt.subplot(1, 2, 1), plt.imshow(np.concatenate([image_CMYK[:,:,i] for i in range(4)], axis=0)), plt.axis('off'), plt.title('Your CMY Channels')
_ = plt.subplot(1, 2, 2), plt.imshow(np.concatenate([image_CMYK_ref[:,:,i] for i in range(4)], axis=0)), plt.axis('off'), plt.title('Reference CMY Channels')

### Section 1.3. Other Features From Color
While using primary lights or colors can be helpful in many situations, there are other representations which are sometimes used in their stead.

Perhaps the most well-known is the HSI, or HSI representation. In this representation, *H* stands for hue, or the shade of color. *S* stands for saturation, which can be seen as a measure of how *pure* the color is. By decreasing the saturation of a color, you mix more of the Black/Grey/White spectrum with it. Finally, *I*, stands for intensity, i.e. how bright the color is.

They formulae for the conversion of a color from the RGB space to HSI are as follows:

$
H = 
\begin{cases}
\theta & \text{if} B \leq G \\
2 \pi - \theta & \text{if} B > G
\end{cases}
$

$
\theta = cos^{-1} \Big \{ \Large \frac{2 R-G-B}{2 \sqrt{(R-G)^2 + (R-B)(G-B)}} \Big \}
$

$
S=1 - \Large \frac{3 min(R,G,B)}{R+G+B}
$

$
I = (R+G+B) / 3
$

While the equations for finding $\theta$ and $H$ might seem hard to grasp, there is a simple way of visualizing how these equations work. The HSI model is based off of a circular view of the color space, where the three colors of red, green, and blue are three vectors which can be used to create the various color within the space. You can see a visualization of this model below. Note the position of three primary colors in the coordinates. The *hue* parameter is simply the angle of the color vector compared to the vector of the color red.

![Hue Space](figures/hue_vectors.jpg "Hue Space")

Now using this knowledge, implement functions for converting between RGB and HSI color spaces.

**Note:** As you might have noticed, hue is essentially a vector angle. Therefore it is generally expressed in degrees. Here, a mapping of $[0 \: 1]$ is done for the hue values. You might see hue values expressed in degrees, radians, or in the 8-bit space of *uint8*.

In [ ]:
def rgb2hsi(image : np.ndarray) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        A color image in np.ndarray format with dtype=np.float64.
    Returns:
    - output : np.ndarray
        The original image, with channels converted from RGB to HSI.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Converting the image colors
# image_HSI = rgb2hsi(image_RGB_ref)
image_HSI_ref = rgb2hsiRef(image_RGB_ref)

# Showing the result
_ = plt.figure(figsize=(18, 18))
_ = plt.subplot(1, 2, 1), plt.imshow(np.concatenate([image_HSI[:,:,i] for i in range(3)], axis=0)), plt.axis('off'), plt.title('Your HSI Channels')
_ = plt.subplot(1, 2, 2), plt.imshow(np.concatenate([image_HSI_ref[:,:,i] for i in range(3)], axis=0)), plt.axis('off'), plt.title('Reference HSI Channels')

The noise pattern that you see on the baboon's nose in the hue channel, is due to the "angle-like" nature of this channel. In other words, a slightly yellowish red will have a very low hue value, while the hue value of a slightly blue-ish red will be close to maximum. Hence the black and white pattern in the hue channel.

Converting from HSI to RGB is a slightly more complicated process, and involves different equations for each of the 3 parts in the hue diagram.
For the **RG** part of the hue, i.e. the part where hue is below $120^{\circ}$, we have:

$
R = I [ 1 + \Large \frac{S cos H}{cos(60^{\circ} - H)} \normalsize]
$

$
B = I (1 - S)
$

$
G = 3 I - R - B
$

And similar functions for the **GB** and **BR** sections of the hue chart. However, in those regions the values of $H$ are subtracted by $120^{\circ}$ and $240^{\circ}$, and the order of the **RGB** channels in the equations above is also altered. Try to rationalize these equations and implement a function to convert the HSI image above back to RGB.

In [ ]:
def hsi2rgb(image : np.ndarray) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        A color image in np.ndarray format with dtype=np.float64.
    Returns:
    - output : np.ndarray
        The original image, with channels converted from HSI to RGB.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Converting the image colors
image_RGB_new = rgb2hsi(image_HSI)
image_RGB_new_ref = hsi2rgbRef(image_HSI_ref) 

# Showing the result
_ = plt.figure(figsize=(14, 12))
_ = plt.subplot(1, 2, 1), plt.imshow(np.concatenate([image_RGB_new[:,:,i] for i in range(3)], axis=0)), plt.axis('off'), plt.title('Your Rebuilt RGB Channels')
_ = plt.subplot(1, 2, 1), plt.imshow(image_RGB_new_ref), plt.axis('off'), plt.title('Reference Rebuilt RGB Channels')

# Scratchpad
You can use this section to try out different codes, without making a mess of the notebook. :)